# Azure Sentinel API Calls

MSTICpy versions > 0.8.5

### Description

This Notebook provides an example of using the Azure Sentinel API features of MSTICpy in order retrieve specific data from Azure Sentinel

### Installation and imports

In [ ]:
!pip install msticpy --upgrade

In [ ]:
from msticpy.data.azure_sentinel import AzureSentinel
import msticpy.nbtools.nbwidgets as widgets

### Authentication
The first step to be able to use the features is to call the AzureSentinel class and connect to it. Authentication uses the standardized Azure authentication options of using environment variables, Azure CLI credentials, Managed Identities, and interactive logons.

In [ ]:
azs = AzureSentinel()
azs.connect()

Once connected we need to select an Azure Sentinel workspace to get details from. The easies way to do this is with the get_subscriptions() and get_sentinel_workspaces() functions to select the subscription and workspace you with to connect to. If you already know which workspace you wish to connect to you can skip straight to the other functions and enter these details.

In [ ]:
subs = azs.get_subscriptions()
print("Select a subscription:")
sub = widgets.SelectItem(item_list=subs['Display Name'].tolist(), auto_display=True)

In [ ]:
sub_id = subs[subs['Display Name'] == sub.value]['Subscription ID'].iloc[0]
workspaces = azs.get_sentinel_workspaces(sub_id)
print("Select an Azure Sentinel Workspace:")
ws = widgets.SelectItem(item_dict=workspaces, auto_display=True)

Now that we have selected our workspace we can call various functions to get details about content in the workspace. These are typically returned as DataFrames. Below we get a list of hunting queries configured in our workspace.

In [ ]:
queries = azs.get_hunting_queries(ws.value)
queries.head()

Hunting queries return the raw queries associated with them, this allows us to pass the query directly to a QueryProvider in order to get the results of the hunting query within the notebook.

In [ ]:
from msticpy.data.data_providers import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig
qry_prov = QueryProvider('LogAnalytics')
wkspace = WorkspaceConfig()
qry_prov.connect(wkspace.code_connect_str)
qry_prov.exec_query(queries['Query'].iloc[2])

We can also get a list of configured alert rules:

In [ ]:
alert_rules = azs.get_alert_rules(ws.value)
alert_rules.head()

We can also get a list of saved bookmarks. To see the events these bookmarks relate to you can pass the query value to a QueryProvider.

In [ ]:
bkmarks = azs.get_bookmarks(ws.value)
bkmarks.head()